<a href="https://colab.research.google.com/github/kaasyapv/Computer_Vision/blob/main/Computer_Vision_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y libzbar0
!pip install pyzbar opencv-python-headless gradio pandas

import cv2
import numpy as np
from pyzbar.pyzbar import decode
import gradio as gr
from datetime import datetime
import pandas as pd
import re

attendance_dict = {}
teacher_logged_in = False

# --- QR scanning function ---
def qr_reader(image):
    global attendance_dict

    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    qr_info = decode(img)
    decoded_texts = []
    message = ""

    for qr in qr_info:
        data = qr.data.decode('utf-8').strip()
        rect = qr.rect
        decoded_texts.append(data)

        # Draw bounding box
        cv2.rectangle(img, (rect.left, rect.top),
                      (rect.left + rect.width, rect.top + rect.height),
                      (30, 144, 255), 3)  # 🔵 blue rectangle

        #  Validate for attendance (only one word names)
        if re.fullmatch(r"[A-Za-z]+", data):
            if data not in attendance_dict:
                attendance_dict[data] = datetime.now().strftime("%H:%M:%S")
                message = f" Attendance marked for {data} at {attendance_dict[data]}"
            else:
                message = f" Duplicate punch detected for {data} (already marked at {attendance_dict[data]})"
        else:
            message = f"ℹ Scanned data: '{data}' — Invalid QR (not a name for attendance)."

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if not decoded_texts:
        message = "No QR code detected."

    return img_rgb, "\n".join(decoded_texts) if decoded_texts else "-", message


# --- Teacher login/logout ---
def teacher_login(password):
    global teacher_logged_in
    if password == "123":
        teacher_logged_in = True
        return gr.update(visible=True), gr.update(visible=False), "Login successful."
    else:
        return gr.update(visible=False), gr.update(visible=True), "Incorrect password."


def teacher_logout():
    global teacher_logged_in
    teacher_logged_in = False
    return gr.update(visible=False), gr.update(visible=True), "Logged out successfully."


# --- Attendance view/reset ---
def view_attendance():
    if attendance_dict:
        df = pd.DataFrame(list(attendance_dict.items()), columns=["Student Name", "Punch-in Time"])
        return df.to_string(index=False)
    return "No attendance data available."


def clear_attendance():
    global attendance_dict
    attendance_dict.clear()
    return "Attendance list cleared."


# --- Dark Blue Theme with Modern Font ---
theme = gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="gray",
    neutral_hue="gray",
    font="IBM Plex Sans"
).set(
    body_background_fill="*neutral_900",
    background_fill_primary="*neutral_800",
    background_fill_secondary="*neutral_900"
)

# --- Interface ---
with gr.Blocks(theme=theme) as demo:
    gr.Markdown("<h2 style='text-align:center;'>QR Code Attendance System</h2>")
    gr.Markdown("<p style='text-align:center;'>Upload a QR code to mark attendance. Teachers can log in to view or manage records.</p>")

    with gr.Row():
        # --- Student Panel ---
        with gr.Column(scale=1):
            gr.Markdown("### Student Panel")
            image_input = gr.Image(type="pil", label="Upload QR Code")
            output_image = gr.Image(label="Scanned Image")
            decoded_text = gr.Textbox(label="Decoded Data", interactive=False)
            msg_box = gr.Textbox(label="System Message", interactive=False)
            scan_btn = gr.Button("Scan QR Code")

            scan_btn.click(fn=qr_reader, inputs=image_input, outputs=[output_image, decoded_text, msg_box])

        # --- Teacher Panel ---
        with gr.Column(scale=1):
            gr.Markdown("### Teacher Panel")

            login_section = gr.Group(visible=True)
            with login_section:
                password_input = gr.Textbox(label="Password", type="password")
                login_btn = gr.Button("Login")
                login_status = gr.Textbox(label="Status", interactive=False)

            teacher_section = gr.Group(visible=False)
            with teacher_section:
                view_btn = gr.Button("View Attendance")
                clear_btn = gr.Button("Clear Attendance")
                logout_btn = gr.Button("Logout")
                attendance_box = gr.Textbox(label="Attendance Records", lines=12, interactive=False)

                view_btn.click(fn=view_attendance, outputs=attendance_box)
                clear_btn.click(fn=clear_attendance, outputs=attendance_box)
                logout_btn.click(fn=teacher_logout, outputs=[teacher_section, login_section, login_status])

            login_btn.click(fn=teacher_login, inputs=password_input, outputs=[teacher_section, login_section, login_status])

demo.launch(debug=True)













Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libzbar0 is already the newest version (0.23.92-4build2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://afb3165e94024b96c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",